# LLaMa 2: A Chatbot LLM on IPUs - Inference

|  Domain | Tasks | Model | Datasets | Workflow |   Number of IPUs   | Execution time |
|---------|-------|-------|----------|----------|--------------|--------------|
|   NLP   |  Chat Fine-tuned Text Generation  | LLaMa 2 7B/13B | N/A | Inference | recommended: 16 (minimum 4) |  20 min  |

[LLaMa 2](https://about.fb.com/news/2023/07/llama-2/) is the next generation of the [LLaMa](https://ai.meta.com/blog/large-language-model-llama-meta-ai/) model by [Meta](https://ai.meta.com/), released as a series of multi-billion parameter models fine-tuned for dialogue. LLaMa 2 was pre-trained on 2 trillion tokens (40% more than the original LLaMa) and shows better performance on benchmarks for equivalent parameter sizes than other SOTA LLMs such as Falcon and MPT.

This notebook will show you how to run LLaMa 2 7B and LLaMa 2 13B models on Graphcore IPUs. In this notebook, we describe how to create and configure the LLaMa inference pipeline, then run live inference as well as batched inference using your own prompts. 

You will need a minimum of 4 IPUs to run this notebook. You can also use 16 IPUs for faster inference using some extra tensor parallelism.

[![Join our Slack Community](https://img.shields.io/badge/Slack-Join%20Graphcore's%20Community-blue?style=flat-square&logo=slack)](https://www.graphcore.ai/join-community)



## Environment setup

If you are running this notebook on Paperspace Gradient, the environment will already be set up for you.

To run the demo using other IPU hardware, you need to have the Poplar SDK enabled. Refer to the [Getting Started guide](https://docs.graphcore.ai/en/latest/getting-started.html#getting-started) for your system for details on how to enable the Poplar SDK. Also refer to the [Jupyter Quick Start guide](https://docs.graphcore.ai/projects/jupyter-notebook-quick-start/en/latest/index.html) for how to set up Jupyter to be able to run this notebook on a remote IPU machine.

In order to improve usability and support for future users, Graphcore would like to collect information about the applications and code being run in this notebook. The following information will be anonymised before being sent to Graphcore:

* User progression through the notebook
* Notebook details: number of cells, code being run and the output of the cells
* Environment details

You can disable logging at any time by running %unload_ext graphcore_cloud_tools.notebook_logging.gc_logger from any cell.

Run the next cell to install extra requirements for this notebook and load the logger.

In [ ]:
%pip install -r requirements.txt
%load_ext graphcore_cloud_tools.notebook_logging.gc_logger

LLaMa 2 is open source and available to use as a Hugging Face checkpoint, but requires access to be granted by Meta. If you do not yet have permission to access the checkpoint and want to use this notebook, [please request access from the LLaMa 2 Hugging Face Model card](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf).

Once you have access, you must be logged onto your Hugging Face Hub account from the Hugging Face CLI in order to load the pre-trained model:

In [ ]:
import huggingface_hub
huggingface_hub.notebook_login()

Next, we define the number of IPUs for your instance as well as the cache directory for the generated executable:

In [ ]:
import os

number_of_ipus = int(os.getenv("NUM_AVAILABLE_IPU", 16))
executable_cache_dir = os.path.join(os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "./exe_cache"), "llama2")
os.environ["POPXL_CACHE_DIR"] = executable_cache_dir

## LLaMa 2 inference

First, load the inference configuration for the model. There are a few configurations made available that you can use in the `config/inference.yml` file. There are different configurations based on model size and the number of available IPUs. You can edit the next cell to choose your model size. It must be one of `7b` or `13b` as both of these are supported.

In [ ]:
model_size = '7b' #or '13b'

In [ ]:
from utils.setup import llama_config_setup

checkpoint_name = f"meta-llama/Llama-2-{model_size}-chat-hf"    
config, *_ = llama_config_setup(
    "config/inference.yml", 
    "release", 
    f"llama2_{model_size}_pod4" if number_of_ipus == 4 else f"llama2_{model_size}_pod16"
)

These names are then used to load the configuration - the function will automatically select and load a suitable configuration for your instance. It will also set the name of the Hugging Face checkpoint to load the model weights and tokenizer.

In [ ]:
config

Next, instantiate the inference pipeline for the model. Here, you simply need to define the maximum sequence length and maximum micro batch size. When executing a model on IPUs, the model is compiled into an executable format with frozen parameters. If these parameters are changed, a recompilation will be triggered.

Selecting longer sequence lengths or batch sizes uses more IPU memory. This means increasing one may require you to decrease the other.

In [ ]:
import api
import time

sequence_length = 1024
micro_batch_size = 2

start = time.time()

llama_pipeline = api.LlamaPipeline(
    config, 
    sequence_length=sequence_length, 
    micro_batch_size=micro_batch_size,
    hf_llama_checkpoint=checkpoint_name
)

print(f"Model preparation time: {time.time() - start}s")

Now you can simply run the pipeline:

In [ ]:
answer = llama_pipeline("Hi, can you tell me something interesting about cats? List it as bullet points.")


Be warned, you may find the model occasionally hallucinates or provides logically incoherent answers. This is expected from a model of this size and you should try to provide prompts which are as informative as possible. Spend some time tweaking the parameters and the prompt phrasing to get the best results you can!

There are a few sampling parameters we can use to control the behaviour of the generation:

- `temperature` – Indicates whether you want more creative or more factual outputs. A value of `1.0` corresponds to the model's default behaviour, with relatively higher values generating more creative outputs and lower values generating more factual answers. `temperature` must be at least `0.0` which means the model picks the most probable output at each step. If the model starts repeating itself, try increasing the temperature. If the model starts producing off-topic or nonsensical answers, try decreasing the temperature.
- `k` – Indicates that only the highest `k` probable tokens can be sampled. Set it to 0 to sample across all possible tokens, which means that top k sampling is disabled. The value for `k` must be between a minimum of 0 and a maximum of `config.model.embedding.vocab_size` which is 32000.
- `output_length` – Indicates the number of tokens to sample before stopping. Sampling can stop early if the model outputs `2` (the end of sequence (EOS) token).
- `print_final` – If `True`, prints the total time and throughput.
- `print_live` – If `True`, the tokens will be printed as they are being sampled. If `False`, only the answer will be returned as a string.
- `prompt` – A string containing the question you wish to generate an answer for.

These can be freely changed and experimented with in the next cell to produce the behaviour you want from the LLaMa 2 model.

In [ ]:
answer = llama_pipeline(
    "How do I get help if I am stuck on a deserted island?",
    temperature=0.4,
    k=10,
    output_length=None,
    print_live=True,
    print_final=True,
)

You can set the `micro_batch_size` parameter to be higher during pipeline creation, and use the pipeline on a batch of prompts. Simply pass the list of prompts to the pipeline, ensuring the number of prompts is less than or equal to the micro batch size.

In [ ]:
prompt = [
    "What came first, the chicken or the egg?",
    "How do I make an omelette with cheese, onions and spinach?",
]
answer = llama_pipeline(
    prompt,
    temperature=0.6,
    k=5,
    output_length=None,
    print_live=False,
    print_final=True,
)

for p, a in zip(prompt, answer):
    print(f"Instruction: {p}")
    print(f"Response: {a}")

LLaMa was trained with a specific prompt format and system prompt to guide model behaviour. This is common with instruction and dialogue fine-tuned models. The correct format is essential for getting sensible outputs from the model. To see the full system prompt and format, you can call the `last_instruction_prompt` attribute on the pipeline.

This is the default prompt format described in this [Hugging Face blog post](https://huggingface.co/blog/llama2#how-to-prompt-llama-2):

In [ ]:
print(llama_pipeline.prompt_format)

Remember to detach your pipeline when you are finished to free up resources:

In [ ]:
llama_pipeline.detach()

That's it! You have now successfully run LLaMa 2 for inference on IPUs.

## Next steps

Check out the full list of [IPU-powered Jupyter Notebooks](https://www.graphcore.ai/ipu-jupyter-notebooks) to see how IPUs perform on other tasks.